After trying a lot of different things, I have put two models in two different files, this one without optimization techniques and the second one with optimization techniques.

I initially used the original Resnet model but then reduced the number of layers. Also, the model was not learning at all so I have to use Batch Normalization in both the models.

In [ ]:
! pip install -q kaggle
from google.colab import files

In [ ]:
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

In [ ]:
! kaggle datasets download -d gpiosenka/100-bird-species/birds

In [ ]:
! mkdir train
! unzip 100-bird-species.zip -d train

In [ ]:
import numpy as np
import torch

I used batch-size of 64 while it was originally 256 in Resnet paper.

In [ ]:
from torchvision import datasets
from torchvision import transforms as transforms
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
traindata = datasets.ImageFolder('train/birds/train', transform=transform)
trainloader=torch.utils.data.DataLoader(traindata,batch_size=64,shuffle=True)
testdata = datasets.ImageFolder('train/birds/test', transform=transform)
testloader=torch.utils.data.DataLoader(traindata,batch_size=64,shuffle=True)

In [ ]:
gpu=torch.cuda.is_available()
gpu

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv0=nn.Conv2d(3,64,7,2,3)
    self.bn0=nn.BatchNorm2d(64)
    self.conv11=nn.Conv2d(64,64,3,padding=1)
    self.bn11=nn.BatchNorm2d(64)
    self.conv12=nn.Conv2d(64,64,3,padding=1)
    self.short1=nn.Identity()
    self.bn12=nn.BatchNorm2d(64)
    self.conv21=nn.Conv2d(64,128,3,2,1)
    self.bn21=nn.BatchNorm2d(128)
    self.conv22=nn.Conv2d(128,128,3,padding=1)
    self.short2=nn.Conv2d(64,128,1,2)
    self.bn22=nn.BatchNorm2d(128)
    self.conv31=nn.Conv2d(128,256,3,2,1)
    self.bn31=nn.BatchNorm2d(256)
    self.conv32=nn.Conv2d(256,256,3,padding=1)
    self.short3=nn.Conv2d(128,256,1,2)
    self.bn32=nn.BatchNorm2d(256)
    self.conv41=nn.Conv2d(256,512,3,2,1)
    self.bn41=nn.BatchNorm2d(512)
    self.conv42=nn.Conv2d(512,512,3,padding=1)
    self.short4=nn.Conv2d(256,512,1,2)
    self.bn42=nn.BatchNorm2d(512)
    self.mpool=nn.MaxPool2d(3,2)
    self.pool=nn.AvgPool2d(7)
    self.fc=nn.Linear(512,275)
    

  def forward(self,x):
    x=F.relu(self.bn0(self.conv0(x)))
#size=112
    x=self.mpool(x)
#size=56
    for i in range(1):
      idx=self.short1(x)
      x=F.relu(self.bn11(self.conv11(x)))
      x=self.bn12(self.conv12(x))
      x=F.relu(x+idx)
#size=56
    idx=self.short2(x)
    x=F.relu(self.bn21(self.conv21(x)))    
    x=self.bn22(self.conv22(x))
    x=F.relu(x+idx)
    for i in range(1):
      idx=self.short1(x)
      x=F.relu(self.bn22(self.conv22(x)))
      x=self.bn22(self.conv22(x))
      x=F.relu(x+idx)
#size=28
    idx=self.short3(x)
    x=F.relu(self.bn31(self.conv31(x)))
    x=self.bn32(self.conv32(x))
    x=F.relu(x+idx)
    for i in range(1):
      idx=self.short1(x)
      x=F.relu(self.bn32(self.conv32(x)))
      x=self.bn32(self.conv32(x))
      x=F.relu(x+idx)
#size=14
    idx=self.short4(x)
    x=F.relu(self.bn41(self.conv41(x)))
    x=self.bn42(self.conv42(x))
    x=F.relu(x+idx)
    for i in range(1):
      idx=self.short1(x)
      x=F.relu(self.bn42(self.conv42(x)))
      x=self.bn42(self.conv42(x))
      x=F.relu(x+idx)
#size=7
    x=self.pool(x)
#size=1
    x=torch.flatten(x,1)
#size=512
    #print(x.shape)
    x=self.fc(x)
    x=F.log_softmax(x,dim=1)
    #print(x[0:4,0:4])
    return x

In [ ]:
model=Net()
print(model)
if gpu:
  model.cuda()
from torch import optim as optim
criterion=nn.NLLLoss()
optimizer=optim.SGD(model.parameters(),lr=0.01)

Net(
  (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv11): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv12): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (short1): Identity()
  (bn12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv21): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn21): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv22): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (short2): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2))
  (bn22): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv31): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), pad

In [ ]:
epochs=8
ploss=float('inf')
for e in range(epochs):
  model.train()
  i=0
  for images,classes in trainloader:
    i=i+1
    if gpu:
      images,classes=images.cuda(),classes.cuda()
    optimizer.zero_grad()
    output=model(images)
    loss=criterion(output,classes)
    loss.backward()
    optimizer.step()
    if i==600:
      break
  ttloss=0
  model.eval()
  for images,classes in testloader:
    if gpu:
      images,classes=images.cuda(),classes.cuda()
    output=model(images)
    loss=criterion(output,classes)
    ttloss+=loss.item()
  ttloss=ttloss/len(testloader)
  print("Test loss epoch",e+1,":",ttloss)
  if ttloss<ploss:
    ploss=ttloss
    torch.save(model.state_dict(), 'resnet03.pt')

Test loss epoch 1 : 4.45321835945179
Test loss epoch 2 : 4.06216389173037
Test loss epoch 3 : 3.6756689087911085
Test loss epoch 4 : 3.86742931681794
Test loss epoch 5 : 3.545668075611065
Test loss epoch 6 : 3.342251757135639
Test loss epoch 7 : 3.444317351688038
Test loss epoch 8 : 3.310756290500814


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
model_save_name = 'resnet03.pt'
path = F"/content/gdrive/MyDrive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [ ]:
optimizer=optim.SGD(model.parameters(),lr=0.0001)

In [ ]:
epochs=4
for e in range(epochs):
  model.train()
  i=0
  for images,classes in trainloader:
    i=i+1
    if gpu:
      images,classes=images.cuda(),classes.cuda()
    optimizer.zero_grad()
    output=model(images)
    loss=criterion(output,classes)
    loss.backward()
    optimizer.step()
    if i==600:
      break
  ttloss=0
  model.eval()
  for images,classes in testloader:
    if gpu:
      images,classes=images.cuda(),classes.cuda()
    output=model(images)
    loss=criterion(output,classes)
    ttloss+=loss.item()
  ttloss=ttloss/len(testloader)
  print("Test loss epoch",e+9,":",ttloss)
  if ttloss<ploss:
    ploss=ttloss
    torch.save(model.state_dict(), 'resnet03.pt')

Test loss epoch 9 : 3.4217023950118524
Test loss epoch 10 : 3.2041431735088297
Test loss epoch 11 : 3.055013313695982
Test loss epoch 12 : 3.2094886302948


In [ ]:
model_save_name = 'resnet03.pt'
path = F"/content/gdrive/MyDrive/{model_save_name}" 
torch.save(model.state_dict(), path)

Till the 12 epochs, Ithe minimum training loss was 3.05 (epoch 11) which would have given accuracy of about 50%.

